<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/03_Sentiment_Analysis/LLama_Colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU info

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jun 29 16:57:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   53C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Connecting to G-drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing and importing libraries

In [ ]:
# Install necessary libraries
!pip install -q -U langchain langchain_community transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.0 MB/s eta 0:00:00


In [ ]:
import os
import json
import pandas as pd
import torch
from tqdm import tqdm
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import HfFolder, HfApi
from google.colab import userdata
import re
import time

# Hugging Face Using Secret Token

In [ ]:
# Retrieve the token from Colab secrets
hf_token = userdata.get('HF_TOKEN')
if hf_token:
    HfFolder.save_token(hf_token)
    api = HfApi()
    user_info = api.whoami()
    if user_info:
        print("Connection to Hugging Face was successful.")
    else:
        print("Failed to connect to Hugging Face. Please check your token.")
else:
    print("Hugging Face token not found. Please set the HF_TOKEN environment variable.")


Connection to Hugging Face was successful.


# Connecting to GitHub Repo

In [ ]:
# Import GitHub token with Google secrets and clone the repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone  https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git


Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 2254, done.
remote: Counting objects: 100% (659/659), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 2254 (delta 416), reused 588 (delta 357), pack-reused 1595
Receiving objects: 100% (2254/2254), 470.85 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.
Updating files: 100% (1163/1163), done.


# LLama code

In [ ]:
TEMPLATE = """<s>Classify the following article into categories with sentiment (Positive, Neutral, Negative, N/A if not applicable) and provide the output in the specified dictionary format.
Example:
Article: ExxonMobil announced a significant increase in quarterly profits due to rising oil prices and increased production levels.
Output: {{'Finance': 'Positive', 'Production': 'Positive', 'Reserves / Exploration / Mergers': 'Neutral', 'Environment / Regulatory': 'Neutral', 'Alternative Energy': 'Neutral', 'Oil/Gas Prices': 'Positive'}}

Example:
Article: Chevron plans to invest heavily in renewable energy projects, aiming to reduce its carbon footprint over the next decade.
Output: {{'Finance': 'Neutral', 'Production': 'Neutral', 'Reserves / Exploration / Mergers': 'Neutral', 'Environment / Regulatory': 'Positive', 'Alternative Energy': 'Positive', 'Oil/Gas Prices': 'Neutral'}}

Example:
Article: BP faced regulatory challenges in its latest drilling project, delaying operations and increasing costs.
Output: {{'Finance': 'Negative', 'Production': 'Negative', 'Reserves / Exploration / Mergers': 'Negative', 'Environment / Regulatory': 'Negative', 'Alternative Energy': 'Neutral', 'Oil/Gas Prices': 'Neutral'}}


Article: {article}

Output only the EXACT dictionary format:
{{'Finance': '[Sentiment]', 'Production': '[Sentiment]', 'Reserves / Exploration / Mergers': '[Sentiment]', 'Environment / Regulatory': '[Sentiment]', 'Alternative Energy': '[Sentiment]', 'Oil/Gas Prices': '[Sentiment]'}}
</s>"""

In [ ]:


def process_article(article, model, tokenizer, generation_pipeline):
    """
    Processes an article using a language model to classify it into predefined categories with sentiment analysis.

    Args:
        article (str): The text of the article to be processed.
        model: The language model used for text generation and classification.
        tokenizer: Tokenizer used to encode the input text for the model.
        generation_pipeline: Pipeline or method for generating text outputs from the model.

    Returns:
        dict or str: A dictionary containing categorized sentiments ('Positive', 'Neutral', 'Negative', or 'N/A' for not applicable)
                    for predefined categories ('Finance', 'Production', 'Reserves / Exploration / Mergers', 'Environment / Regulatory',
                    'Alternative Energy', 'Oil/Gas Prices'), or a message indicating no valid JSON output found after 5 attempts.
    """

    for attempt in range(5):  # Attempt to find valid JSON 5 times
        # Create a prompt using the template
        prompt = TEMPLATE.format(article=article)

        # Encode the prompt and truncate to max length
        inputs = tokenizer(prompt, max_length=1024, truncation=True, return_tensors='pt')

        # Move inputs to the correct device
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate the classification using the LLaMA model
        result = model.generate(**inputs, max_new_tokens=200, num_beams=2, early_stopping=True)

        # Decode the result
        generated_text = tokenizer.decode(result[0], skip_special_tokens=True)

        # Split the response to separate the input prompt and the output
        split_response = generated_text.split("</s>", 1)
        if len(split_response) > 1:
            final_response = split_response[1].strip()
        else:
            final_response = generated_text.strip()

        # Extract the JSON part from the final response
        output_start = final_response.find("Output:")
        if output_start != -1:
            output_text = final_response[output_start:]
            json_output = re.search(r"\{.*\}", output_text)
            if json_output:
                try:
                    return json.loads(json_output.group(0).replace("'", '"'))
                except json.JSONDecodeError:
                    continue  # Continue to next attempt if JSON decoding fails
        else:
            continue  # Continue to next attempt if no "Output:" is found

    return "No valid JSON output found after 5 attempts."


# Model and Pipeline settings

In [ ]:
# Load the LLaMA model and tokenizer
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    quantization_config=quantization_config,
    use_auth_token=hf_token
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", use_auth_token=hf_token)

# Create text generation pipeline
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=8192,  # increase to max token size supported by the model
    truncation=True,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Data processing

In [ ]:
# Path to the CSV file containing articles
#invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
#invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
#earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
#earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
#sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = proquest_df
#text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(3635, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,PQ-2703147472,CVX,2022-08-17,Oil Giants Must Face Climate-Liability Suits i...,Chevron Corp.\nSpending projections are slidin...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,PQ-2702197510,CVX,2022-08-15,Warren Buffett's Berkshire Hathaway Keeps Spen...,Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,PQ-2697049245,CVX,2022-08-02,"Investors Put Forward More Proposals, Dialing ...",Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,PQ-2696371267,CVX,2022-07-30,Soaring Fuel Prices Catapult Oil Giants to Rec...,SENATORS aired misgivings on Tuesday over pote...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,PQ-2695835797,CVX,2022-07-29,"Exxon, Chevron, Shell Report Record Profits on...",LONDON—Royal Dutch Shell PLC said it would sta...,https://www.proquest.com/newspapers/shell-trie...


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
3630,ProQuest,PQ-2238653651,PSX,NaN,"As the Price of Gas Tumbles, Even California I...",Gasoline prices have been falling fast in rece...,https://www.proquest.com/newspapers/as-price-g...
3631,ProQuest,PQ-2239478009,PSX,NaN,"Phillips 66, Plains plan pipeline system",CUSHING Phillips 66 and Plains All American Pi...,https://www.proquest.com/newspapers/phillips-6...
3632,ProQuest,PQ-2238860015,PSX,NaN,Autonomous robot inspects in-service diesel st...,"HOUSTON Phillips 66, Square Robot Inc. and Ver...",https://www.proquest.com/newspapers/autonomous...
3633,ProQuest,PQ-2233473767,PSX,NaN,United States/Mexico: Report: Trump's Tariff o...,US President Donald Trump's threats to tax Mex...,https://www.proquest.com/newspapers/united-sta...
3634,ProQuest,PQ-2232566793,PSX,NaN,Trump's Mexico Tariffs Could Hit Americans at ...,President Trump's\nproposed tariffs on Mexico\...,https://www.proquest.com/newspapers/trumps-mex...


In [ ]:
# Path to the intermediate output file
intermediate_output_file = "/content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_intermediate.json"
final_output_file = "/content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_final.json"

# Name of the column containing the article text
article_column = 'Article Text'

# Output dictionary to store results
output_dict = {}

# Check if the intermediate output file exists and load it
if os.path.exists(intermediate_output_file):
    with open(intermediate_output_file, "r") as file:
        output_dict = json.load(file)
    # Find the ID of the last processed article in the output_dict
    last_processed_id = list(output_dict.keys())[-1]
    # Find the index of the last processed article in the DataFrame
    start_index = text_df[text_df['Unique_ID'] == last_processed_id].index[0] + 1
else:
    # If intermediate output file doesn't exist, start processing from the beginning
    start_index = 0

# Process each article in the DataFrame starting from the last saved state
for index, row in tqdm(text_df.iloc[start_index:].iterrows(), total=len(text_df) - start_index):
    unique_id = row['Unique_ID']
    article = row[article_column]

    output = process_article(article, model, tokenizer, generation_pipeline)
    output_dict[unique_id] = output
    print(f"Processed article {unique_id}")

    # Save the output dictionary to a JSON file every 1 iterations
    if (index + 1) % 1 == 0:
        with open(intermediate_output_file, "w") as file:
            json.dump(output_dict, file, indent=4)
        print(f"Intermediate results saved to {intermediate_output_file}")

# Save the final output dictionary to a JSON file
with open(final_output_file, "w") as file:
    json.dump(output_dict, file, indent=4)

print(f"Final results saved to {final_output_file}")

  0%|          | 0/760 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 1/760 [02:40<33:55:32, 160.91s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed article PQ-2884523343
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_intermediate.json


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 2/760 [05:21<33:48:41, 160.58s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed article PQ-2884103743
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_intermediate.json


  0%|          | 3/760 [05:53<21:24:07, 101.78s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Processed article PQ-2238860015
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_intermediate.json


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|█████████▉| 759/760 [20:08:35<02:24, 144.31s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed article PQ-2233473767
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_intermediate.json


100%|██████████| 760/760 [20:08:54<00:00, 95.44s/it] 

Processed article PQ-2232566793
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_intermediate.json
Final results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/output_results_final.json
